# SQL Alchemy With Python
## Introduction
The code samples provided here showcase some basic functionality within sqlalchemy and pandas, including:
* How to read straight into pandas
* How to convert from connection execution statements into pandas
* Use cases of when you can use each example

## Requirements
This tutorial just uses an in memory database with no external connections, but if you want to connect to an Oracle db you will need an ODBC driver downloaded. I THINK you can find instructions on how to do that for Windows, here: https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html#installing-cx-oracle-on-windows

 * Python environment able to import the dependencies as listed below

## Setup
In this part we'll configure some sample dataframes to use as a demonstration

In [1]:
import pandas as pd

from sqlalchemy import create_engine, text

In [3]:
#In memory connection for purpose of this guide:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

#An actual connection could look more like the following:
#engine = create_engine('oracle+cx_oracle://%s:%s@%s' % (USERNAME, PASSWORD, CON_STRING), echo=True) 

In [23]:
#If you need to drop the tables at any point, uncomment this and run:
#with engine.connect() as conn:
    #conn.execute("DROP TABLE some_table")
    #conn.execute("DROP TABLE other_table")

In [25]:
#Dataframe 1
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
# Adding values to a table
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )


2023-01-30 20:22:49,359 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-01-30 20:22:49,366 INFO sqlalchemy.engine.Engine [cached since 839.6s ago] ()
2023-01-30 20:22:49,369 INFO sqlalchemy.engine.Engine COMMIT
2023-01-30 20:22:49,373 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-01-30 20:22:49,377 INFO sqlalchemy.engine.Engine [cached since 839.6s ago] ((1, 1), (2, 4))
2023-01-30 20:22:49,377 INFO sqlalchemy.engine.Engine COMMIT
2023-01-30 20:22:49,378 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-30 20:22:49,378 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-01-30 20:22:49,379 INFO sqlalchemy.engine.Engine [cached since 839.6s ago] ((6, 8), (9, 10))
2023-01-30 20:22:49,379 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
#Dataframe 2
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE other_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO other_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
# Adding values to a table
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO other_table (x, y) VALUES (:x, :y)"),
        [{"x": 5, "y": 8}, {"x": 9, "y": 11}], #Difference on x, and y values compared to dataframe 1
    )

2023-01-30 20:22:42,490 INFO sqlalchemy.engine.Engine CREATE TABLE other_table (x int, y int)
2023-01-30 20:22:42,494 INFO sqlalchemy.engine.Engine [generated in 0.00444s] ()
2023-01-30 20:22:42,504 INFO sqlalchemy.engine.Engine COMMIT
2023-01-30 20:22:42,510 INFO sqlalchemy.engine.Engine INSERT INTO other_table (x, y) VALUES (?, ?)
2023-01-30 20:22:42,512 INFO sqlalchemy.engine.Engine [generated in 0.00150s] ((1, 1), (2, 4))
2023-01-30 20:22:42,512 INFO sqlalchemy.engine.Engine COMMIT
2023-01-30 20:22:42,513 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-30 20:22:42,513 INFO sqlalchemy.engine.Engine INSERT INTO other_table (x, y) VALUES (?, ?)
2023-01-30 20:22:42,514 INFO sqlalchemy.engine.Engine [cached since 0.003707s ago] ((5, 8), (9, 11))
2023-01-30 20:22:42,514 INFO sqlalchemy.engine.Engine COMMIT


# Examples

## Directly read with Pandas
Useful with:
* small table reads
* when you need to return something with certainty as a dataframe

In [26]:
sql = """
select x, y from some_table
"""
with engine.connect() as conn:
    df = pd.read_sql(sql, conn)

df.tail()

2023-01-30 20:23:05,765 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("
select x, y from some_table
")
2023-01-30 20:23:05,770 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-30 20:23:05,773 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("
select x, y from some_table
")
2023-01-30 20:23:05,779 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-30 20:23:05,780 INFO sqlalchemy.engine.Engine 
select x, y from some_table

2023-01-30 20:23:05,781 INFO sqlalchemy.engine.Engine [raw sql] ()


,x,y
0,1,1
1,2,4
2,6,8
3,9,10


## Reading connection execution result into pandas
Useful with:
* When you need to run a SQL statement without capturing data (i.e a procedure or function to generate another table)
* If data gets larger
* You need to pass it through any kind of custom pre-processing

In [27]:
sql2 = """
select x, y from other_table
"""
with engine.connect() as conn:
    result = conn.execute(sql2)
    df2 = pd.DataFrame(result.fetchall())
    df2.columns = result.keys()

df2.tail()

2023-01-30 20:23:07,649 INFO sqlalchemy.engine.Engine 
select x, y from other_table

2023-01-30 20:23:07,656 INFO sqlalchemy.engine.Engine [raw sql] ()


,x,y
0,1,1
1,2,4
2,5,8
3,9,11


## Comparing query results
In this section we will look at basic way of comparing dataframes.
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.compare.html


In [30]:
df.compare(df2)

x           y      
  self other  self other
2  6.0   5.0   NaN   NaN
3  NaN   NaN  10.0  11.0